# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

> It uses sliding window view instead of sliding window for splitting the dataset

In [1]:
from dvats.all import *
from tsai.data.preparation import prepare_forecasting_data
from tsai.data.validation import get_long_term_forecasting_splits, get_forecasting_splits
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Get configuration parameters from yml
> Configuration parameters are obtained from 'config\03-embeddings.yaml'

### Get configuration artifact

In [2]:
import utils.config as cfg
config, job_type = cfg.get_artifact_config_embeddings(print_flag = False)

### Show configuration artifact

In [3]:
for key, value in config.items():
    print(f"{key}: {value}")

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/mvp-SWV:latest
input_ar: None
cpu: False


## Build W&B artifact

In [4]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="03b_embeddings-sliding_window_view"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

runname: 03b_embeddings-sliding_window_view


In [5]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow',
    name        = runname
)

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


'stream.Stream' object attribute 'write' is read-only


## Get trained model artifact

### Build artifact selector
> Botch to use artifacts offline

In [6]:
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

### Get the model from W&B
> Restore the encoder model and its associated configuration

In [7]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [8]:
print("enc_artifact: "+enc_artifact.name)

enc_artifact: mvp-SWV:v3


In [9]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

wandb:   1 of 1 files downloaded.  


## Get dataset artifact from W&B
### Restore the dataset artifact used for training the encoder. 
> Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that it matches with the dataset that we want to reduce.

In [10]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'solar_4_seconds:v0'

### Specify the dataset artifact that we want to get the embeddings from
> If no artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [11]:
input_ar_name = ifnone(
    config.input_ar, 
    f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}'
)
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'solar_4_seconds:v0'

In [12]:
df = input_ar.to_df()
df.head()

wandb: Downloading large artifact solar_4_seconds:v0, 112.87MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.3


,0
2019-08-01 00:00:03,0.0
2019-08-01 00:00:07,0.0
2019-08-01 00:00:11,0.0
2019-08-01 00:00:15,0.0
2019-08-01 00:00:19,0.0


In [13]:
df.shape

(7397222, 1)

### Split data with Sliding Window

In [14]:
w = enc_run.config['w']
stride = enc_run.config['stride']
enc_input, _ = prepare_forecasting_data(df, fcst_history = w)
enc_input = enc_input[::stride]
enc_input.shape

(493144, 1, 75)

### Get embedings

In [15]:
enc_learner.dls.bs = enc_run.config['batch_size']

In [ ]:
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

--> Use CUDA |Get enc embs GPU 
CUDA está disponible
Dispositivo CUDA actual:  0
Nombre del dispositivo CUDA actual:  NVIDIA GeForce RTX 3090
--> Set dataset from X (enc_learn does not contain dls)
--> Get module
--> Get enc embs bs:  1024


In [ ]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings-SWV', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [ ]:
run.finish()